This script is used to generate the blue-to-red light gradeint direction vs. light/dark side, see Figure 4 and Figure 5 of Schallock & Hayes 2024
* x axis [-1,1] where values are distance-from-center/semi-minor-len. Positive numbers refer to dark sides per Iye. et al. 2019 and Negative number refer to 'bright side' (opposite of dark side).
* y axis [0,1] where 0 is the pixel of reddest value in difference image, 1 is the pixel of bluest value in difference image
* Only includes pixels within 45 degrees of minor axis
* Excludes trendline whose fitting likely failed (has min(y) <= -0.5 or max(y) >= 1.5), as well as cases where gofher label and spin parity labels are orthogonal.

In [109]:
import os

import sys
sys.path.insert(0, '../gofher')

import numpy as np
import matplotlib.pyplot as plt
import math
from collections import defaultdict

from gofher import run_gofher
from matrix import create_dist_matrix, create_centered_mesh_grid, create_minor_axis_angle_matrix, create_major_axis_angle_matrix
from spin_parity import read_spin_parity_galaxies_label_from_csv, standardize_galaxy_name, score_label


In [110]:
survery_to_use = "panstarrs"

BANDS_IN_ORDER = ['g','r','i','z','y'] #Important: Must stay in order of BLUEST to REDDEST Waveband (Editting this will cause gofher to no longer correctly evaluate redder side of galaxy)
REF_BANDS_IN_ORDER = ['i','z','y','r','g'] #The prefernce each waveband being choosen as refernce band from highest priority to lowest priority

In [111]:
#for formatting mnras paper: source: https://walmsley.dev/posts/typesetting-mnras-figures

SMALL_SIZE = 9
MEDIUM_SIZE = 9
BIGGER_SIZE = 9

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)
#plt.rc('font', family='Nimbus Roman No9 L') #uncomment this to use same font as MNRAS - requires font Nimbus Roman No9 L (can be downloaded from https://www.fontsquirrel.com/fonts/nimbus-roman-no9-l)

In [112]:
path_to_catalog_data = "..\\..\\spin-parity-catalog-data"
output_path_for_gradient_trend = "..\\..\\..\\gradient_vs_dark_side.png"

In [113]:
def get_galaxies(figure_to_run_on):
    return os.listdir(os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on))

In [114]:
def get_dark_side_csv_path(folder_name):
    csv_path = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog-data\\catalog"
    return os.path.join(csv_path,"{}.csv".format(folder_name))

def get_color_image_path(name,folder_name):
    return os.path.join(path_to_catalog_data,folder_name,name,"{}_color.jfif".format(name))

In [115]:
def seperate_out_for_gradient(the_gal, paper_label, poly_degree, max_ang_from_minor,linspace=100):
    x = the_gal.gofher_params.x; y = the_gal.gofher_params.x
    theta = the_gal.gofher_params.theta
    shape = the_gal[the_gal.ref_band].data.shape

    xv,yv = create_centered_mesh_grid(x,y,shape)
    dist = create_dist_matrix(xv,yv)
    ang = np.abs(create_minor_axis_angle_matrix(x,y,theta,shape)) #for near minor-axis
    #ang = np.abs(create_major_axis_angle_matrix(x,y,theta,shape)) #for near major-axis
    #ang[ang > max_ang_from_minor] = 0.0

    el_mask = the_gal.create_ellipse()
    pos_mask, neg_mask = the_gal.create_bisection() #for near minor-axis
    #pa = the_gal.gofher_params #for near major-axis
    #pa.theta += np.pi/2 #for near major-axis
    #pos_mask, neg_mask = the_gal.create_bisection(pa) #for near major-axis

    to_include = np.logical_and(el_mask,ang < max_ang_from_minor)
    b = the_gal.gofher_params.b #for near minor-axis
    #b = the_gal.gofher_params.a #for near major-axis

    
    to_return = dict()

    for band_pair_key in the_gal.band_pairs:
        band_pair = the_gal.get_band_pair(band_pair_key)
        diff_image = band_pair.diff_image
        diff_image[np.logical_not(to_include)] = -np.Inf

        ones = np.ones(shape)
        score = score_label(band_pair.classification_label, paper_label)
        paper_classification = band_pair.classification*score*-1 #*x-1 for issue with flip

        if paper_classification == 1:
            ones[pos_mask] *= -1
        elif paper_classification == -1:
            ones[neg_mask] *= -1
        else:
            return None

        xs = ((ones*dist)/b)[np.logical_not(np.isinf(diff_image))]
        ys = diff_image[np.logical_not(np.isinf(diff_image))]
        
        the_min = np.min(diff_image[np.logical_not(np.isinf(diff_image))])
        the_max = np.max(diff_image[np.logical_not(np.isinf(diff_image))])
        ys_normed = (ys-the_min)/(the_max-the_min)

        to_return[band_pair_key] = np.polynomial.Polynomial.fit(xs, ys_normed, poly_degree).linspace(linspace)

    return to_return

def plot_all_trends(all_trends, output_path="", poly_degree=10, linspace=100):
    keys = [['g-r', 'g-i'],
            ['g-z', 'g-y'],
            ['r-i', 'r-z'],
            ['r-y','i-z'],
            ['i-y','z-y']]
    first_fig = ['a','b','c','d','e','f','g','h','i','j']
    count = 0
    fig, axd = plt.subplot_mosaic(keys,figsize=(10/3, 7),
                                  constrained_layout=True,num=1, clear=True) #num=1, clear=True #https://stackoverflow.com/a/65910539/13544635
    fig.patch.set_facecolor('white')
    for band_pair in all_trends:
        xs = []
        ys = []
        for each in all_trends[band_pair]:
            x = each[0]
            y = each[1]
            mask = np.logical_and(x>-1.0,x<1.0)
            xs.extend(list(x[mask]))
            ys.extend(list(y[mask]))
            axd[band_pair].plot(x,y, alpha=0.25,linewidth=1.0,c='blue')

        aggregate_trend = np.polynomial.Polynomial.fit(xs, ys, poly_degree).linspace(linspace)
        axd[band_pair].plot(*aggregate_trend,linewidth=1.0,c='red', ls='--')

        axd[band_pair].axvline(0.0,c='black')

        
        i = np.argmin(aggregate_trend[1])
        min_x = aggregate_trend[0][i]
        axd[band_pair].axvline(min_x,c='red',linewidth=1.0)
        
        the_title = "({}) {}:".format(first_fig[count],band_pair)
        axd[band_pair].set_title(the_title)
        axd[band_pair].set_ylabel("||Difference||")
        axd[band_pair].set_xlabel("||Distance||")
        
        axd[band_pair].set_xlim([-1, 1])
        axd[band_pair].set_ylim([0, 1])
        count += 1
    if output_path != "":
        fig.savefig(output_path, dpi = 300, bbox_inches='tight')
        fig.clear()
        plt.close(fig)
    else:
        plt.show()


In [116]:
def run_on_folder_name(folder_name):
    paper_labels = read_spin_parity_galaxies_label_from_csv(get_dark_side_csv_path(folder_name))

    all_trends = defaultdict(list)

    i=1
    for name in get_galaxies(folder_name):
        print(i,name)

        if standardize_galaxy_name(name) not in paper_labels:
            print("skippimg",name)
            continue

        paper_label = paper_labels[standardize_galaxy_name(name)]

        get_fits_path = lambda name,band: os.path.join(path_to_catalog_data,survery_to_use,folder_name,name,"{}_{}.fits".format(name,band))

        try:
            the_gal = run_gofher(name,get_fits_path,BANDS_IN_ORDER,REF_BANDS_IN_ORDER, paper_label)
            gal_trend_lines = seperate_out_for_gradient(the_gal,paper_label,10,math.pi/4)
            #gal_trend_lines = seperate_out_for_gradient(the_gal,paper_label,10,math.pi/12)
            
            if isinstance(gal_trend_lines,type(None)):
                print("skipping")
                continue
            to_include = True
            for each_band_pair in gal_trend_lines:
                x = gal_trend_lines[each_band_pair][0]
                y = gal_trend_lines[each_band_pair][1]
                if np.max(x) > 2.0 or np.min(x) < -2.0 or np.max(y) > 1.5 or np.min(y) < -0.5:
                    to_include = False
                    break

            if not to_include:
                print("skipping")
                continue

            for each_band_pair in gal_trend_lines:
                all_trends[each_band_pair].append(gal_trend_lines[each_band_pair])

        except Exception as e:
            print("Exception on gal",name,e)
        i += 1
        #if i > 2: break #you can uncomment this if you are debugging something and just want to run on a single galaxy
    return all_trends


In [117]:
observed_folders = ["figure8","figure10","figure11"] #use this for galaxies in which dark side was directly observed (see Iye et al. 2019)
inferred_folders = ["figure9"] #use this for galaxies in which dark side was infered (see Iye et al. 2019)

In [118]:
save_figure = True

In [119]:
if not os.path.exists(path_to_catalog_data):
    raise ValueError("The path to the catalog is not found {} - make sure you update path_to_catalog_data".format(path_to_catalog_data))

all_trends = defaultdict(list)

#for folder_name in observed_folders:
for folder_name in inferred_folders:
    print(folder_name)
    returned = run_on_folder_name(folder_name)
    for each_key in returned:
        all_trends[each_key].extend(returned[each_key])

output_pa = output_path_for_gradient_trend if save_figure else ""
plot_all_trends(all_trends,output_pa,10,100)


figure9
1 IC 4566
2 IC1151
3 IC1199
4 IC1256
5 IC1528
6 IC2095
7 IC2163
skipping
7 IC2487
8 IC467
9 IC5309
10 IC674
11 IC776
skipping
11 IRAS03056+2034
12 NGC1
13 NGC1012
14 NGC1042
15 NGC1058
skipping
15 NGC1068
16 NGC1073
skipping
16 NGC1087
skipping
16 NGC1169
17 NGC1232
18 NGC1300
19 NGC1358
20 NGC160
21 NGC1614
22 NGC1645
23 NGC1677
24 NGC171
25 NGC1744
26 NGC177
27 NGC1784
28 NGC180
skipping
28 NGC1832
29 NGC192
30 NGC2139
31 NGC214
32 NGC217
33 NGC2207
34 NGC2276
skipping
34 NGC23
35 NGC2336
36 NGC234
37 NGC237
38 NGC2449
39 NGC2486
40 NGC2487
41 NGC2500
42 NGC2525
43 NGC2540
44 NGC2552
Error fitting sersic, using inital_gofher_parameters from sep
45 NGC2553
46 NGC257
47 NGC2604
skipping
47 NGC2608
48 NGC2730
49 NGC2776
50 NGC278
51 NGC2805
52 NGC289
Exception on gal NGC289 read_fits: Fits at file path not found ..\..\spin-parity-catalog-data\panstarrs\figure9\NGC289\NGC289_g.fits
53 NGC2906
54 NGC2916
55 NGC2964
skipping
55 NGC2998
56 NGC3057
skipping
56 NGC3106
57 NGC3145
58 N

c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Error fitting sersic, using inital_gofher_parameters from sep
178 NGC7691
179 NGC7716
180 NGC7723
181 NGC7741
skipping
181 NGC7743
182 NGC776
183 NGC7819
184 NGC7824
185 NGC853
186 NGC864
skipping
186 NGC895
187 NGC922
188 NGC925
189 PGC02162
skipping
189 PGC03512
190 PGC05673
skipping
190 PGC06855
191 PGC07826
192 PGC08941
193 PGC14564
194 PGC15531
skipping
194 PGC16274
195 PGC19767
skipping
195 PGC20938
196 PGC23333
197 PGC23598
skipping
197 PGC23913
198 PGC24788
199 PGC26140
200 PGC26517
201 PGC27792
202 PGC28310
203 PGC28401
skipping
203 PGC31159
204 PGC32091
skipping
204 PGC32638
205 PGC33465
206 PGC36925
skipping
206 PGC38268
207 PGC38908
208 PGC39728
209 PGC46767
skipping
209 PGC49906
210 PGC55750
211 PGC56010
212 PGC57931
skipping
212 PGC58410
skipping
212 PGC71106
213 PGC72144
214 PGC72453
215 UGC10310
216 UGC10796
217 UGC1081
218 UGC10811
219 UGC1087
skipping
219 UGC11262
220 UGC11318
skipping
220 UGC12224
221 UGC12274
222 UGC12308
223 UGC12391
224 UGC12494
225 UGC12816
226 U

c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


262 UGC807
263 UGC8196
264 UGC8516
265 UGC8733
266 UGC8781
267 UGC9067
268 UGC9177
269 UGC9476


c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\school\Desktop\github\gofher\gofher_env\Lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Error fitting sersic, using inital_gofher_parameters from sep
270 UGC9837
271 UGC9965


In [120]:
k = list(all_trends.keys())[0]
print(len(all_trends[k]))

269
